In [7]:
cd /home/jupyter/PACE

/home/jupyter/PACE


In [2]:
ls

 GCP_ML4Floods_intersection.ipynb              intersections.ipynb
'ML4Floods Demo.ipynb'                         intersections_sketchpad.ipynb
 ML4Floods_BuildingFootprint_Reduction.ipynb   query_functions.ipynb
 ML4Floods_FootprintandFloodMap.ipynb          query_sketchpad.ipynb
 footprint_reduction.ipynb                     query_workflow_test.ipynb


# **Install & import packages**

In [3]:
!pip install git+https://github.com/spaceml-org/ml4floods#egg=ml4floods
!pip install pygeos

  Cloning https://github.com/spaceml-org/ml4floods to /tmp/pip-install-rrsym5c_/ml4floods_9ae443af60ef474ba37f349ea32afa3b
  Running command git clone --filter=blob:none --quiet https://github.com/spaceml-org/ml4floods /tmp/pip-install-rrsym5c_/ml4floods_9ae443af60ef474ba37f349ea32afa3b
  Resolved https://github.com/spaceml-org/ml4floods to commit 0f8fdc4b6b891ed249f52fda19ee23c50462e237
  Preparing metadata (setup.py) ... done


In [4]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import pygeos
gpd.options.use_pygeos = True

# **Getting the buildings affected by the floods in the government supplied AOI.**

## Getting the intersection DF

In [64]:
# Returns a geodata with the buildings affected in the government supplied AOI.

def get_intersection(aoi_footprintPath: str, aoi_mapPath: str ):
    
    # Reading the files in.
    aoi_footprint = gpd.read_file(aoi_footprintPath)
    aoi_map = gpd.read_file(aoi_mapPath)
    
    aoi_floodTrace= aoi_map.groupby('class').get_group('flood-trace')
    aoi_footprint['footprint_index'] = aoi_footprint.index
    
    # Finding the affected buildings in the government supplied AOI. 
    aoi_intersect = gpd.overlay(aoi_floodTrace, aoi_footprint, how='intersection')
    
    aoi_footprint['footprint_index'] = aoi_footprint.index
    
    intersection = aoi_footprint.loc[aoi_intersect['footprint_index']]
    intersection = intersection.drop(columns= 'footprint_index')

    length = len(intersection)
    
    # Number of affected buildings.
    print(f'There are {length} buildings affected in this AOI')
    
    # Saving the file. 
    intersection.to_file("data/aoi_intersect.geojson", driver='GeoJSON')
    
    return intersection

In [48]:
aoi_footprint_path = 'data/syd_reduced_footprint.geojson'
aoi_footprint = gpd.read_file(aoi_footprint_path)

aoi_map_path = 'data/syd_reduced_map.geojson'
aoi_map = gpd.read_file(aoi_map_path)

In [65]:
test_intersection = get_intersection(aoi_footprint_path, aoi_map_path)
test_intersection.head()

There are 399 buildings affected in this AOI


,geometry
14713,"POLYGON ((150.80703 -32.48325, 150.80711 -32.4..."
20010,"POLYGON ((150.80881 -32.29166, 150.80882 -32.2..."
14697,"POLYGON ((150.80611 -32.29109, 150.80614 -32.2..."
10710,"POLYGON ((150.80792 -32.28899, 150.80807 -32.2..."
14729,"POLYGON ((150.80753 -32.28908, 150.80757 -32.2..."


In [66]:
test_intersection.explore()

# NEW 

In [50]:
aoi_floodTrace= aoi_map.groupby('class').get_group('flood-trace')

Storing the index for both DFs

In [53]:
aoi_footprint['footprint_index'] = aoi_footprint.index

In [55]:
aoi_footprint.head()

,geometry,footprint_index
0,"POLYGON ((150.80033 -32.32067, 150.80037 -32.3...",0
1,"POLYGON ((150.81640 -32.30648, 150.81642 -32.3...",1
2,"POLYGON ((150.83105 -32.73790, 150.83114 -32.7...",2
3,"POLYGON ((150.84422 -32.44826, 150.84432 -32.4...",3
4,"POLYGON ((150.81490 -32.25455, 150.81503 -32.2...",4


In [43]:
aoi_floodTrace.head()

,class,geometry,intersection_index
125,flood-trace,"POLYGON ((150.77920 -32.45335, 150.77920 -32.4...",125
126,flood-trace,"POLYGON ((150.81227 -32.59426, 150.81165 -32.5...",126
127,flood-trace,"POLYGON ((150.80230 -32.59264, 150.80124 -32.5...",127
128,flood-trace,"POLYGON ((150.81061 -32.57484, 150.81062 -32.5...",128
129,flood-trace,"POLYGON ((150.80853 -32.57282, 150.80907 -32.5...",129


In [21]:
test3 = gpd.overlay(aoi_floodTrace, aoi_footprint, how='intersection')

In [14]:
test3.head()

,class,intersection_index,footprint_index,geometry
0,flood-trace,137,14713,"POLYGON ((150.80711 -32.48319, 150.80714 -32.4..."
1,flood-trace,157,20010,"POLYGON ((150.80891 -32.29164, 150.80892 -32.2..."
2,flood-trace,159,14697,"POLYGON ((150.80607 -32.29101, 150.80606 -32.2..."
3,flood-trace,160,10710,"POLYGON ((150.80794 -32.28899, 150.80792 -32.2..."
4,flood-trace,160,14729,"POLYGON ((150.80761 -32.28906, 150.80765 -32.2..."


In [42]:
intersection = aoi_footprint.loc[test3['footprint_index']]
intersection = intersection.drop(columns= 'footprint_index')
intersection.head()

,geometry
14713,"POLYGON ((150.80703 -32.48325, 150.80711 -32.4..."
20010,"POLYGON ((150.80881 -32.29166, 150.80882 -32.2..."
14697,"POLYGON ((150.80611 -32.29109, 150.80614 -32.2..."
10710,"POLYGON ((150.80792 -32.28899, 150.80807 -32.2..."
14729,"POLYGON ((150.80753 -32.28908, 150.80757 -32.2..."


In [ ]:
intersection.to_file("data/aoi_intersect.geojson", driver='GeoJSON')

# end of notebook

TO DO

PERMANENT WATER LAYER

CHECKING ON PROGRESS OF DOWNLOADS 

CHECKING ON PROGRESS OF MAPPING: gonzalo help with tracking the progress?

DM (digital elevation models) for flood depth

mentors mentioned that they previously plotted in plt but will use explore() from now on (beth found it)